# Comparación entre Dask y Polars para Procesamiento de Grandes Volúmenes de Datos en Python

En este notebook, vamos a comparar las capacidades de las bibliotecas Dask y Polars para el manejo de grandes volúmenes de datos en Python. Ambas bibliotecas ofrecen operaciones paralelizadas y están diseñadas para manejar grandes datasets de manera eficiente, pero difieren en sus enfoques, APIs y estrategias de optimización.

Objetivos

	•	Comprender las diferencias entre Dask y Polars.
	•	Comparar su rendimiento en tareas comunes de procesamiento de datos.
	•	Evaluar el uso de memoria, velocidad de cómputo y facilidad de uso para cada biblioteca.

## Configuración

Instalación de Bibliotecas Requeridas

In [1]:
import dask.dataframe as dd
import polars as pl
import pandas as pd
import time
import numpy as np
from dask.distributed import LocalCluster, Client
import os

## Crear cluster

In [ ]:
# Crear un cluster local con múltiples workers y threads
cluster = LocalCluster(n_workers=4, threads_per_worker=2, dashboard_address = '8880')  # 4 workers, cada uno con 2 threads
client = Client(cluster)

# Mostrar el dashboard de Dask para monitoreo 
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8880/status,
Dashboard: http://127.0.0.1:8880/status,Workers: 4
Total threads: 8,Total memory: 8.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:61452,Workers: 4
Dashboard: http://127.0.0.1:8880/status,Total threads: 8
Started: Just now,Total memory: 8.00 GiB
Comm: tcp://127.0.0.1:61465,Total threads: 2
Dashboard: http://127.0.0.1:61469/status,Memory: 2.00 GiB
Nanny: tcp://127.0.0.1:61455,


## Cargar datos

In [ ]:
# Medir el tiempo de carga del dataset
start = time.time()

# Cargar el dataset usando Dask 
dask_df = dd.read_csv(
    'data/large_dataset.csv',
    blocksize="64MB"  # Ajusta el tamaño de cada partición
)

end = time.time()
print(f"Tiempo de carga con Dask (con cluster): {end - start:.2f} segundos")

print("Primeras 5 filas del DataFrame de Dask:")
dask_df.head(5) 


Tiempo de carga con Dask (con cluster): 0.05 segundos
Primeras 5 filas del DataFrame de Dask:


,id,value1,value2
0,0,0.077351,0.988993
1,1,0.020837,0.429316
2,2,0.504242,0.756492
3,3,0.830112,0.079156
4,4,0.631661,0.162702


In [4]:
start = time.time()
polars_df = pl.read_csv('data/large_dataset.csv')
end = time.time()
print(f"Tiempo de carga con Polars: {end - start:.2f} segundos")
polars_df.head(5)

Tiempo de carga con Polars: 0.32 segundos


id,value1,value2
i64,f64,f64
0,0.077351,0.988993
1,0.020837,0.429316
2,0.504242,0.756492
3,0.830112,0.079156
4,0.631661,0.162702


## Comparación de Procesamiento de Datos

Vamos a realizar algunas tareas comunes de procesamiento de datos para comparar el rendimiento de Dask y Polars. Realizaremos operaciones como filtrado, agregación y agrupación en los datos.

### Tarea 1: Filtrado de Datos

In [ ]:
# ===============================
# FILTRADO CON DASK
# ===============================

# Persistir el DataFrame en memoria distribuida
dask_df = dask_df.persist()  

start = time.time()

# Realizar el filtrado
filtered_dask_df = dask_df[dask_df['value1'] > 0.5]  # Operación diferida
filtered_result = filtered_dask_df.compute()  # Forzar la ejecución y traer el resultado

end = time.time()
print(f"Tiempo de filtrado con Dask (optimizado): {end - start:.2f} segundos")

# ===============================
# FILTRADO CON Polars
# ===============================

start = time.time()
filtered_polars_df = polars_df.filter(pl.col('value1') > 0.5)
end = time.time()
print(f"Tiempo de filtrado con Polars: {end - start:.2f} segundos")

Tiempo de filtrado con Dask (optimizado): 0.63 segundos
Tiempo de filtrado con Polars: 0.03 segundos


### 2: Agrupación y Agregación

In [8]:
# Agrupar por 'id' módulo 10 y calcular el promedio de 'value1'
start = time.time()
agg_dask_df = dask_df.groupby(dask_df['id'] % 10).value1.mean().compute()
end = time.time()
print(f"Tiempo de agregación por grupo con Dask: {end - start:.2f} segundos")
agg_dask_df

Tiempo de agregación por grupo con Dask: 0.14 segundos


id
0    0.499636
1    0.499607
2    0.501343
3    0.498651
4    0.500604
5    0.500397
6    0.498977
7    0.500246
8    0.500528
9    0.500268
Name: value1, dtype: float64

In [9]:
# Agrupar por 'id' módulo 10 y calcular el promedio de 'value1'
start = time.time()
agg_polars_df = (
    polars_df
    .with_columns((pl.col("id") % 10).alias("grupo_id"))  # Agregar la columna 'grupo_id'
    .group_by("grupo_id")  # Agrupar por 'grupo_id'
    .agg(pl.col("value1").mean().alias("mean_value1"))  # Calcular el promedio de 'value1' en cada grupo
)
end = time.time()

# Imprimir el tiempo de ejecución y el resultado
print(f"Tiempo de agregación por grupo con Polars: {end - start:.2f} segundos")
agg_polars_df

Tiempo de agregación por grupo con Polars: 0.03 segundos


grupo_id,mean_value1
i64,f64
1,0.499607
3,0.498651
8,0.500528
0,0.499636
5,0.500397
9,0.500268
4,0.500604
7,0.500246
6,0.498977


### 3. Operación de Unión (Join)

In [ ]:
# Crear un DataFrame más pequeño para unir con el dataset principal
small_dask_df = dask_df.head(1000)

start = time.time()
joined_dask_df = dask_df.merge(small_dask_df, on='id', how='inner').compute()
end = time.time()
print(f"Tiempo de unión con Dask: {end - start:.2f} segundos")
joined_dask_df

Tiempo de unión con Dask: 0.05 segundos


,id,value1_x,value2_x,value1_y,value2_y
0,0,0.077351,0.988993,0.077351,0.988993
1,1,0.020837,0.429316,0.020837,0.429316
2,2,0.504242,0.756492,0.504242,0.756492
3,3,0.830112,0.079156,0.830112,0.079156
4,4,0.631661,0.162702,0.631661,0.162702
...,...,...,...,...,...
995,995,0.228302,0.075440,0.228302,0.075440
996,996,0.884049,0.256424,0.884049,0.256424
997,997,0.992169,0.524715,0.992169,0.524715
998,998,0.525595,0.240857,0.525595,0.240857


In [ ]:
# Crear un DataFrame más pequeño para unir con el dataset principal
small_polars_df = polars_df.head(1000)

start = time.time()
joined_polars_df = polars_df.join(small_polars_df, on='id', how='inner')
end = time.time()
print(f"Tiempo de unión con Polars: {end - start:.2f} segundos")
joined_polars_df

Tiempo de unión con Polars: 0.02 segundos


id,value1,value2,value1_right,value2_right
i64,f64,f64,f64,f64
0,0.077351,0.988993,0.077351,0.988993
1,0.020837,0.429316,0.020837,0.429316
2,0.504242,0.756492,0.504242,0.756492
3,0.830112,0.079156,0.830112,0.079156
4,0.631661,0.162702,0.631661,0.162702
…,…,…,…,…
995,0.228302,0.07544,0.228302,0.07544
996,0.884049,0.256424,0.884049,0.256424
997,0.992169,0.524715,0.992169,0.524715


# ¿Por qué las operaciones de Dask fueron más lentas que las de Polars?

Cuando comparamos Dask y Polars en operaciones como filtrado, agrupación y agregación en un entorno de una sola máquina, es común observar que **Polars es más rápido que Dask**. 

## 1. Implementación en Lenguaje de Bajo Nivel

- **Polars** está implementado en **Rust**, un lenguaje de bajo nivel que permite manejar la memoria y los recursos de la CPU de manera más eficiente que Python.
- Rust optimiza operaciones como el acceso a memoria y el procesamiento de datos en bloques, lo cual es especialmente beneficioso en operaciones de alta concurrencia y uso intensivo de CPU.
- **Dask**, en cambio, está escrito en Python, un lenguaje interpretado y de más alto nivel que tiene algunas limitaciones en cuanto a eficiencia de procesamiento, especialmente en tareas que requieren un manejo muy optimizado de memoria y CPU.

## 2. Diferencias en el Modelo de Ejecución

- **Dask** utiliza un modelo de ejecución **diferida**, donde las operaciones no se ejecutan inmediatamente. En lugar de eso, Dask construye un **gráfico de tareas** (task graph) que organiza las operaciones en un flujo de trabajo optimizado, especialmente útil para entornos distribuidos.
  - Esto implica una **sobrecarga de planificación** que, aunque es útil en operaciones complejas o distribuidas, introduce un retraso adicional en tareas simples en una sola máquina.
- **Polars** puede ejecutar las operaciones de manera **inmediata** en memoria (sin planificación diferida), lo cual permite que las operaciones se ejecuten más rápidamente en situaciones donde la planificación extra de Dask no es necesaria.

## 3. Estructura de Datos Columnares en Polars

- Polars utiliza una **estructura de datos columnares** que está optimizada para operaciones como selección de columnas y filtrado.
- Esta estructura permite a Polars trabajar con datos en bloques de memoria continuos y realizar operaciones columnares de forma muy rápida, ya que no necesita acceder a datos de otras columnas o filas.
- **Dask**, por otro lado, está basado en la estructura de datos de pandas, que está orientada a filas y no está optimizada de la misma manera para operaciones columnares rápidas.

## 4. Menor Necesidad de Manejo de Particiones en Polars

- **Dask** divide el DataFrame en **múltiples particiones** para procesarlas en paralelo. Este enfoque es ideal para datasets que no caben en memoria, ya que permite dividir el trabajo entre varias máquinas o núcleos de CPU.
  - Sin embargo, en operaciones simples, el manejo de particiones añade una sobrecarga. Dask necesita combinar los resultados de cada partición después de realizar la operación, lo cual puede ralentizar el proceso.
- **Polars** trabaja en un solo bloque de datos en memoria (si el dataset cabe en memoria), lo cual evita la sobrecarga de manejar particiones y resulta en operaciones más rápidas en datasets de tamaño moderado que caben en la memoria de una sola máquina.

## 5. Uso de SIMD y Paralelismo Interno en Polars

- **Polars** está diseñado para aprovechar instrucciones **SIMD (Single Instruction, Multiple Data)**, que permiten procesar múltiples elementos en paralelo a nivel de CPU, optimizando así operaciones como el filtrado y la agregación.
- **Dask** paraleliza tareas al nivel de particiones y usa múltiples hilos de Python. Sin embargo, su paralelismo está limitado por el **Global Interpreter Lock (GIL)** de Python, lo que puede reducir el rendimiento en comparación con el procesamiento paralelo optimizado de Polars en Rust.

## 6. Diferentes Casos de Uso

- **Dask** es ideal para entornos distribuidos o cuando se trabaja con datasets que no caben en memoria, ya que permite procesar datos en múltiples máquinas o núcleos de CPU.
- **Polars** está optimizado para cargas de trabajo en una sola máquina y es extremadamente eficiente en operaciones en memoria cuando el dataset cabe en la RAM de la máquina.
- En casos de uso de **big data distribuido**, Dask puede ser más eficiente que Polars debido a su capacidad de escalar horizontalmente, mientras que en operaciones en una sola máquina, Polars suele tener un mejor rendimiento.


**Dask** es excelente para procesamiento distribuido y escalabilidad en entornos donde el dataset es demasiado grande para caber en la memoria de una sola máquina, mientras que **Polars** es ideal para procesamiento rápido en memoria en una máquina única.

# Ejemplos donde Dask es Mejor que Polars

In [ ]:
# Definir el tamaño del archivo y la cantidad de filas
num_rows = 10_000_000  # Aproximadamente 50 GB
chunk_size = 1_000_000  # Tamaño de cada chunk (10 millones de filas)

# Columnas del DataFrame
columns = ['id', 'category', 'value1', 'value2']

# Especificar la carpeta y el nombre del archivo
output_folder = "data"  # Carpeta donde guardar el archivo
output_file = "large_dataset_2.csv"

# Crear la carpeta si no existe
os.makedirs(output_folder, exist_ok=True)

# Ruta completa al archivo
output_path = os.path.join(output_folder, output_file)

# Crear el archivo CSV en chunks
for i in range(0, num_rows, chunk_size):
    # Crear un chunk de datos
    df = pd.DataFrame({
        'id': np.arange(i, i + chunk_size),
        'category': np.random.choice(['A', 'B', 'C', 'D'], size=chunk_size),
        'value1': np.random.rand(chunk_size),
        'value2': np.random.rand(chunk_size)
    })

    # Guardar el chunk en el archivo CSV (modo append después del primer chunk)
    if i == 0:
        # Si es el primer chunk, escribe con encabezado
        df.to_csv(output_path, index=False)
    else:
        # Si no es el primer chunk, añade al archivo sin encabezado
        df.to_csv(output_path, mode='a', header=False, index=False)

    # Imprimir progreso
    print(f"Chunk {i // chunk_size + 1} guardado ({i + chunk_size}/{num_rows} filas procesadas)")

print(f"Archivo CSV grande generado como: {output_path}")

Chunk 1 guardado (1000000/10000000 filas procesadas)
Chunk 2 guardado (2000000/10000000 filas procesadas)
Chunk 3 guardado (3000000/10000000 filas procesadas)
Chunk 4 guardado (4000000/10000000 filas procesadas)
Chunk 5 guardado (5000000/10000000 filas procesadas)
Chunk 6 guardado (6000000/10000000 filas procesadas)
Chunk 7 guardado (7000000/10000000 filas procesadas)
Chunk 8 guardado (8000000/10000000 filas procesadas)
Chunk 9 guardado (9000000/10000000 filas procesadas)
Chunk 10 guardado (10000000/10000000 filas procesadas)
Archivo CSV grande generado como: data/large_dataset_2.csv


In [13]:
# Medir el tiempo de carga del dataset
start = time.time()

# Cargar el dataset usando Dask (ajusta la cantidad de particiones si es necesario)
dask_df2 = dd.read_csv(
    'data/large_dataset_2.csv',
    blocksize="64MB"  # Ajusta el tamaño de cada partición
)

# Puedes cambiar el blocksize segun tu memoria RAM

end = time.time()
print(f"Tiempo de carga con Dask (con cluster): {end - start:.2f} segundos")

# Mostrar las primeras 5 filas del DataFrame
print("Primeras 5 filas del DataFrame de Dask:")
dask_df2.head(5)  # `head()` ejecuta las operaciones necesarias para mostrar estas filas

Tiempo de carga con Dask (con cluster): 0.14 segundos
Primeras 5 filas del DataFrame de Dask:


,id,category,value1,value2
0,0,C,0.228227,0.473876
1,1,C,0.412750,0.370915
2,2,D,0.100294,0.302345
3,3,A,0.391325,0.185428
4,4,A,0.727223,0.584141


In [ ]:
start = time.time()
polars_df2 = pl.read_csv('data/large_dataset.csv')
end = time.time()
print(f"Tiempo de carga con Polars: {end - start:.2f} segundos")
polars_df2.head(5)

Tiempo de carga con Polars: 1.74 segundos


id,category,value1,value2
i64,str,f64,f64
0,"""C""",0.228227,0.473876
1,"""C""",0.41275,0.370915
2,"""D""",0.100294,0.302345
3,"""A""",0.391325,0.185428
4,"""A""",0.727223,0.584141


En este ejemplo, cargamos un conjunto muy grande de datos, y al hacer la partición en chunks con Dask, se paraleliza la carga y es más eficiente.